# Connect to Amazon Redshift with Pyspark, Spark Scala, and SparkR


## Table of Contents:

1. [Prerequisites](#Prerequisites)
2. [Introduction](#Introduction)
3. [Setup](#Setup)
4. [Connect to Amazon Redshift using Pyspark](#Connect-to-Amazon-Redshift-using-Pyspark)
5. [Connect to Amazon Redshift using Scala](#Connect-to-Amazon-Redshift-using-Scala)
6. [Connect to Amazon Redshift using SparkR](#Connect-to-Amazon-Redshift-using-SparkR)


## Prerequisites

In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.
* This example we connect to Amazon Redshift cluster, hence the EMR cluster attached to this notebook must have the connectivity (VPC) and appropriate rules (Security Group).


## Introduction
In this example we use Pyspark, Spark Scala, and Spark R to connect to a table in Amazon Redshift using spark-redshift connector.

[spark-redshift](#https://github.com/spark-redshift-community/spark-redshift) is a performant Amazon Redshift data source for Apache Spark

## Setup

* Create an S3 bucket location to be used as a temporary location for Redshift dataset. For example: s3://EXAMPLE-BUCKET/temporary-redshift-dataset/

* Create an AWS IAM role which will be associated to the Amazon Redshift cluster. Make sure that this IAM role has access to read and write to the above mentioned S3 bucket location with the appropriate IAM policy. More details:

 * [Create AWS IAM role for Amazon Redshift](#https://docs.aws.amazon.com/redshift/latest/gsg/rs-gsg-create-an-iam-role.html)
 * [Associate IAM role with Amazon Redshift cluster](#https://docs.aws.amazon.com/redshift/latest/dg/c-getting-started-using-spectrum-add-role.html)


In [ ]:
%%configure -f
{ 
    "conf": 
        {
            "spark.jars.packages": "org.apache.spark:spark-avro_2.11:2.4.2,io.github.spark-redshift-community:spark-redshift_2.11:4.0.1"
        }
}

## Connect to Amazon Redshift using Pyspark

In [ ]:
%%pyspark

#Declare the variables and replace the variables values as appropiate

str_jdbc_url='jdbc:redshift://<Redshift Endpoint>:5439/dev?user=<username>&password=<password>'
str_dbname='<Database name>'
str_tgt_table='<Target Table>'
str_s3_path='s3://<S3 Path>'
str_iam_role='<IAM Role ARN>'

# Read data from source table

jdbcDF = spark.read \
    .format("io.github.spark_redshift_community.spark.redshift") \
    .option("url", str_jdbc_url) \
    .option("dbtable", str_dbname) \
    .option("tempdir", str_s3_path) \
    .option("aws_iam_role",str_iam_role) \
    .load()

jdbcDF.limit(5).show()

# Write data to target table

jdbcDF.write \
    .format("io.github.spark_redshift_community.spark.redshift") \
    .option("url", str_jdbc_url) \
    .option("dbtable", str_tgt_table) \
    .option("tempdir", str_s3_path) \
    .option("aws_iam_role",str_iam_role).mode('append').save()

## Connect to Amazon Redshift using Scala

In [ ]:
%%scalaspark

#Declare the variables and replace the variables values as appropiate

val str_jdbc_url="jdbc:redshift://<Redshift Endpoint>:5439/dev?user=<username>&password=<password>"
val str_dbname="<database name>"
val str_tgt_table="<Target table>"
val str_s3_path="s3://<S3 Path>"
val str_iam_role="<IAM Role ARN>"
val str_username="<Username>"
val str_password="<Password>"

# Read data from source table
val jdbcDF = (spark.read.format("io.github.spark_redshift_community.spark.redshift")
  .option("url", str_jdbc_url)
  .option("dbtable", str_dbname)
  .option("tempdir", str_s3_path)
  .option("aws_iam_role", str_iam_role)
  .load())

# Write data to target table

jdbcDF.limit(5).show()

jdbcDF.write.mode("append").
    format("io.github.spark_redshift_community.spark.redshift").option("url", str_jdbc_url).option("dbtable", str_tgt_table).option("aws_iam_role", str_iam_role).option("tempdir", str_s3_path).save()
    


## Connect to Amazon Redshift using SparkR

In [ ]:
%%rspark

#Declare the variables and replace the variables values as appropiate

str_jdbc_url='jdbc:redshift://<Redshift Endpoint>:5439/dev?user=<username>&password=<password>'
str_dbname='<Database name>'
str_tgt_table='<Target Table>'
str_s3_path='s3://<S3 Path>'
str_iam_role='<IAM Role ARN>'

# Read data from source table

df <- read.df(
    NULL,
    "io.github.spark_redshift_community.spark.redshift",
    aws_iam_role = str_iam_role,
    tempdir = str_s3_path,
    dbtable = str_src_table,
    url = str_jdbc_url)

showDF(df)